I originally inteded to scrape the official WotC magic.wizards.com site for tournament results but they unfortunately don't make much available outside of very recent events. After some research, tournament data will be scraped from mtgtop8.com, who offer *years* worth of back data. Some individual card data could be scraped here as well, but scryfall.com offers much more robust card data via a convenient API.

interesting note on data gathering: https://www.dailyesports.gg/wizards-of-the-coast-mtg-frank-karsten-stop-publishing-gp-results-win-rates/

In [1]:
import re
import requests
import numpy as np
import pandas as pd
from pprint import pprint
from time import sleep
from bs4 import BeautifulSoup, NavigableString
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

# Connect to MongoDB
client = MongoClient('mongodb://192.168.0.209', 27017)
db = client['capstone_1']
cards_coll = db['cards']
decks_coll = db['decks']
decklists_coll = db['deck_lists']
events_coll = db['events']

# Base URLs for building requests
scryfall_api_url = 'https://api.scryfall.com/{}'  # API docs: https://scryfall.com/docs/api
mtgtop8_url = 'https://www.mtgtop8.com/{}'

# mtgtop8.com format keys for building requests
mt8_format_keys = {
    'vintage': 'VI',
    'legacy': 'LE',
    'modern': 'MO',
    'pioneer': 'PI',
    'historic': 'HI',
    'standard': 'ST',
    'commander': 'EDH',
    'limited': 'LI',
    'pauper': 'PAU',
    'peasant': 'PEA',
    'block': 'BL',
    'extended': 'EX',
    'highlander': 'HIGH',
    'canadian_highlander': 'CHL'
}

def query(link, payload={}):
    """A requests wrapper function"""
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print('WARNING', response.status_code)
        print(response.content)
    return response

def get_card(name_str):
    """Returns data from magicthegathering.io API on an individual card by name"""
    payload = {'name': name_str}
    response = query('https://api.magicthegathering.io/v1/cards', payload)
    return response.json()

def hot_soup(url, payload={}):
    """Makes a steaming bowl of hot soup"""
    response = query(url, payload)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def gather_archtypes(meta_url):
    """Gathering a list of archtypes among deck strategies for a meta"""
    soup = hot_soup(meta_url)
    archtypes = { strat: [] for strat in ['aggro', 'control', 'combo'] }
    for strat in soup.find_all(class_='Stable')[0].find_all(rowspan=True):  # In this table, only the style type headers use 'rowspan'
        strat_str = strat.contents[0].lower()  # Get corrosponding key for archtypes dict
        item = strat.parent

        # Gather each archtype under each strategy type
        while len(archtypes[strat_str]) < int(strat['rowspan']) - 1:  # Rowspan == number of archtypes under this style
            item = item.next_sibling
            if isinstance(item, NavigableString):
                continue
            if item.a:  # If this sibling has a link, we know it's what we're looking for
                text  = item.a.text
                num_decks = int(item.contents[3].text)
                archtypes[strat_str].append((text, num_decks))
    
    return archtypes

def scrape_events(meta_url, collection):
    """Scrape event pages using selenium"""
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)
    driver.get(meta_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    meta_dropdown = soup.find('select', {'name': 'meta'})  # get drop down selector for meta
    selected_meta = meta_dropdown.find('option', selected=True)  # get current meta
    
    def get_next(d, class_name):
        """Check if the next button is still valid"""
        try:
            button = d.find_elements_by_class_name('Nav_PN')[-1]
            return button if button.text == 'Next' else False
        except Exception as e:
            return False
    
    page = 1
    while True:
        print(f'\nScraping event page {page}...')
        next_btn = get_next(driver, 'Nav_PN')
        soup = BeautifulSoup(driver.page_source, 'html.parser')  # make some soup
        
        for event in soup.find_all(class_='Stable')[2].find_all(class_='hover_tr'):  # 10 events list table
            """
                This loop iterates through event table rows, pulling out an ID number,
                the star rating and the date of the event
            """
            link = event.a  # associated hyperlink
            eid = re.search(r"e=(\d+)&", link['href']).group(1)  # unique id number
            stars = event.find(class_='O16').find_all('img')  # star rating / level
            collection.insert_one({
                'id': eid,
                'name': link.text,
                'date': event.find(class_='S10').text,
                'level': 4 if 'bigstar' in stars[0]['src'] else len(stars),
                'link': mtgtop8_url.format(link['href']),
                'meta': selected_meta.text
            })
            
        if next_btn:
            next_btn.click()
            page += 1
            sleep(1)
        else:
            print('\n\n')
            driver.close()
            break

def scrape_top_decks(event_url, event_id, archtypes, collection):
    """Takes in a url to an event page and scrapes the top decks"""
    soup = hot_soup(event_url)
    top8_list = soup.find('div', {'id': 'top8_list'})
    decks_table = top8_list.contents[0] if top8_list else soup.find_all(class_='Stable')[0]
    # num_players = int(re.search(r"(\d+) players -", decks_table.text).group(1))  This fails sometimes but I haven't found the culprit
    for sib in decks_table.next_siblings:
        if isinstance(sib, NavigableString):
            continue
        if sib.a:
            href = sib.a['href']
            did = re.search(r"d=(\d+)&", href).group(1)  # unique id number
            link = mtgtop8_url.format('event' + href)
            
            # Check if this deck already exists - I was getting duplicate values for some reason, but a surprisingly small number of them.
            if collection.count_documents({'id': did}, limit=1) > 0:
                print(f'Duplicate found: {link}')
            
            mainboard, sideboard, archtype = scrape_decklist(link)
            placement, title, pilot = sib.text.split('\n')[1:-1]
            deck = {
                'id': did,
                'event_id': event_id,
                'title': title,
                'pilot': pilot,
                'archtype': archtype,
                'placement': placement,
                'mainboard': mainboard,
                'sideboard': sideboard,
                'link': link
            }
            for strat, arches in archtypes.items():
                if archtype in [arch[0] for arch in arches]:
                    deck['strategy'] = strat
            collection.insert_one(deck)
            print(f"  - {deck['title']} by {deck['pilot']} scraped.")

def scrape_decklist(deck_url):
    """Takes in a deck url and returns mainboard and sideboard cards with their quantities"""
    soup = hot_soup(deck_url)
    card_re = re.compile(r"(\d+)\s(.+)")
    deck_table = soup.find_all(class_='Stable')[1]
    deck_headers = deck_table.previous_sibling.previous_sibling.find_all('td')
    archtype = deck_headers[2].text.replace('decks', '')
    cardlist = deck_table.table.find_all('table')
    mainboard, sideboard = [], []
    for row in cardlist.pop().find_all('span'):
        count, card = card_re.search(row.parent.text).groups()
        sideboard.append((int(count), card.strip()))

    for col in cardlist:
        for row in col.find_all('span'):
            count, card = card_re.search(row.parent.text).groups()
            mainboard.append((int(count), card.strip()))

    return mainboard, sideboard, archtype

def scrape_metas(form):
    """Scrapes all potential metas from a format page"""
    soup = hot_soup(mtgtop8_url.format('format'), {'f': mt8_format_keys[form]})
    meta_dropdown = soup.find('select', {'name': 'meta'})  # get drop down selector for meta
    metas = {opt.text: mtgtop8_url.format(opt['value']) for opt in meta_dropdown.find_all('option')}  # meta URLs
    return metas

In [2]:
def initial_scrape():  # First scrape of events and winning decklists
    form = 'standard'
    meta = 'History - All Worlds'
    metas = scrape_metas(form)
    chosen_meta = metas[meta]  # a meta url will be fed into gather_archtypes() and scrape_events()

    archtypes = gather_archtypes(chosen_meta)
    print(f'Scraping meta [{meta}]...')
    events = scrape_events(chosen_meta, events_coll)
    
    for event in events_coll.find():
        print(f"Scraping decks from event {event['name']}...")
        scrape_top_decks(event['link'], event['id'], archtypes, decks_coll)
        print('\n')

# This function makes a persisting change to the database and has already been run.
# initial_scrape()

Here is where the scraping happens.

In [3]:
deck_df = pd.DataFrame(list(decks_coll.find({}, {'_id':0, 'link':0})))
event_df = pd.DataFrame(list(events_coll.find({}, {'_id':0, 'link':0})))

In [4]:
event_df.head()

,id,name,date,level,meta
0,27955,Red Bull Untapped 2020 World Finals,01/11/20,3,History - All Worlds
1,24631,World Championship XXVI Honolulu,12/02/20,4,History - All Worlds
2,20758,2018 World Magic Cup,16/12/18,4,History - All Worlds
3,20145,Worlds 2018 (Las Vegas),23/09/18,4,History - All Worlds
4,17741,2017 World Magic Cup,03/12/17,4,History - All Worlds


In [5]:
deck_df.head()

,id,event_id,title,pilot,archtype,placement,mainboard,sideboard,strategy
0,421716,27955,Gruul Aggro,Kazune Kosaka,Gruul Aggro,1,"[[4, Cragcrown Pathway], [2, Evolving Wilds], ...","[[2, Embereth Shieldbreaker], [2, Klothys, God...",aggro
1,421717,27955,Abzan Aggro,Koutarou Ishibashi,Abzan Midrange,2,"[[4, Branchloft Pathway], [4, Brightclimb Path...","[[4, Cling to Dust], [2, Containment Priest], ...",aggro
2,421719,27955,Esper DOOM Yorion,Gobetti Enrico,Esper Control,3-4,"[[4, Brightclimb Pathway], [4, Clearwater Path...","[[1, Yorion, Sky Nomad], [2, Archon of Sun's G...",control
3,421718,27955,Gruul Aggro,Luke Hancock,Gruul Aggro,3-4,"[[4, Cragcrown Pathway], [2, Evolving Wilds], ...","[[2, Chainweb Aracnir], [2, Embereth Shieldbre...",aggro
4,421720,27955,Dimir Mill,Michael Bonde,Dimir Control,5-8,"[[4, Clearwater Pathway], [4, Fabled Passage],...","[[1, Lurrus of the Dream-Den], [1, Anticogniti...",control


Some EDA showed that most entries put 3rd and 4th place into a '3-4' category, and likewise with 5th through 8th.</br>
However fewer entries list these places individually, so here we update database records to have consistent groupings.

In [6]:
def placement_fix():
    """Regrouping placement records for easier catigoration"""
    def func(place):
        if place in ['3', '4']:
            return '3-4'
        elif place in ['5', '6', '7', '8']:
            return '5-8'
        else:
            return place
        
    for place in decks_coll.find({}):
        record_id = place['_id']
        decks_coll.update_one({'_id': record_id}, {'$set': {'placement': func(place['placement'])}})

# This function makes a persisting change to the database and has already been run.
# placement_fix()

In [7]:
merged = event_df.merge(deck_df, left_on='id', right_on='event_id')
convert = {s: int for s in ['id_y', 'event_id']}
merged = merged.astype(convert)
merged.set_index('id_y', inplace=True)
merged.rename_axis('id', inplace=True)
merged.drop('id_x', axis=1, inplace=True)
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 421716 to 253152
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       336 non-null    object
 1   date       336 non-null    object
 2   level      336 non-null    int64 
 3   meta       336 non-null    object
 4   event_id   336 non-null    int64 
 5   title      336 non-null    object
 6   pilot      336 non-null    object
 7   archtype   336 non-null    object
 8   placement  336 non-null    object
 9   mainboard  336 non-null    object
 10  sideboard  336 non-null    object
 11  strategy   330 non-null    object
dtypes: int64(2), object(10)
memory usage: 34.1+ KB


Here I noticed the 'strategy' column has null values, we'll fix those below.

In [8]:
merged[merged['strategy'].isnull()]

,name,date,level,meta,event_id,title,pilot,archtype,placement,mainboard,sideboard,strategy
id,,,,,,,,,,,,
207945,Worlds 2010 (Chiba),12/12/10,4,History - All Worlds,770,Eldrazi Green,Love Janse,Card Kingdom$430 (NM),3-4,"[[4, Eldrazi Temple], [1, Eye of Ugin], [12, F...","[[4, Acidic Slime], [1, Eye of Ugin], [3, Obst...",NaN
108523,Worlds 2007 (New York),06/12/07,4,History - All Worlds,117,Mannequin,Kotaro Otsuka,Card Kingdom$423 (NM),3-4,"[[4, Faerie Conclave], [2, Mouth of Ronom], [4...","[[2, Damnation], [2, Liliana Vess], [3, Namele...",NaN
108521,Worlds 2007 (New York),06/12/07,4,History - All Worlds,117,Elves!,Katsuhiro Mori,Card Kingdom$373 (NM),5-8,"[[5, Forest], [4, Gilt-Leaf Palace], [4, Llano...","[[2, Cloudthresher], [1, Krosan Grip], [3, Lil...",NaN
108516,Worlds 2007 (New York),06/12/07,4,History - All Worlds,117,Elves!,Yoshitaka Nakano,Card Kingdom$415 (NM),5-8,"[[5, Forest], [4, Gilt-Leaf Palace], [4, Llano...","[[2, Cloudthresher], [1, Krosan Grip], [3, Lil...",NaN
253183,Worlds 1995 (Seattle) *,01/08/95,4,History - All Worlds,9185,(Sideboard B),Henri Schildt,Card Kingdom$59 (NM),5-8,"[[60, Unknown Card]]","[[4, Circle of Protection: Red], [3, Howling M...",NaN
253184,Worlds 1995 (Seattle) *,01/08/95,4,History - All Worlds,9185,(Sideboard C),Mu Luen Wang,Card Kingdom$98 (NM),5-8,"[[60, Unknown Card]]","[[1, Balance], [1, Black Vise], [1, Circle of ...",NaN


Some more EDA led me to a few entries with incomplete data.</br>
We'll remove some and fix others below.

In [9]:
# Dropping decks with missing lists and correcting strats with missing archtypes
merged.drop([253183, 253184, 108514], axis=0, inplace=True)

strat_fix = {
    'Eldrazi Green': 'aggro',
    'Mannequin': 'control',
    'Elves!': 'aggro'
}

for rid in [207945, 108523, 108521, 108516]:
    title = merged.loc[207945]['title']
    merged.at[rid, 'strategy'] = strat_fix[title]
    merged.at[rid, 'archtype'] = title
    

In [10]:
def build_decklists(df):
    """takes in our merged events dataframe and fills Decklist database"""
    for index, row in df[['mainboard', 'sideboard']].iterrows():
        for i, vals in enumerate(row):
            for qty, card in vals:
                decklists_coll.insert_one({
                    'deck_id': index,
                    'name': card,
                    'quantity': qty,
                    'board': row.index[i]
                })
# This function makes a persisting change to the database and has already been run.
# build_decklists()

In [11]:
decklist_df = pd.DataFrame(list(decklists_coll.find({}, {'_id':0, 'link':0})))

In [13]:
unique_cards = decklist_df['name'].unique()
print(f'{len(unique_cards)} unique cards.')

def scrape_cards(cards_list):
    i = 1
    for name in cards_list:
        cards_coll.insert_one(get_card(name)['cards'][0])
        if i % 10 == 0:
            print(f'{i} cards gathered.')
        i += 1
        sleep(1)

# This function makes a persisting change to the database and has already been run.
# scrape_cards(unique_cards)

1998 unique cards.


I could have made some better choices pulling the card data in from the api.</br>
We'll take some steps to clean it up before and after creating a dataframe below.

In [18]:
remove_keys = [
    'set',
    'setName',
    'artist',
    'number',
    'multiverseid',
    'imageUrl',
    'rulings',
    'foreignNames',
    'printings',
    'originalText',
    'originalType',
    'legalities',
    'variations',
    'watermark'
]

cards_data = list(cards_coll.find({'cards': { '$exists': True, '$ne': [] }}, {'_id': 0}))
cards_data = [datum['cards'][0] for datum in cards_data]

cards_df = pd.DataFrame(cards_data)
cards_df.drop(remove_keys, axis=1, inplace=True)
cards_df.set_index('name', inplace=True)


In [30]:
print('in unique list, but not in database')
print(np.setdiff1d(unique_cards, np.array(cards_df.index)))
print('\n\n')
print('in database, but not in unique')
print(np.setdiff1d(np.array(cards_df.index), unique_cards))

in unique list, but not in database
['Absorb' 'Addle' 'Aladdin' 'Anger' 'Berserk' 'Cataclysm' 'Censor' 'Char'
 'Commit / Memory' 'Compulsion' 'Consign / Oblivion' 'Cut / Ribbons'
 'Discovery / Dispersal' 'Dispatch' 'Dispel' 'Disrupt' 'Eliminate' 'Exile'
 'Expansion / Explosion' 'Explore' 'Find / Finality' 'Fire / Ice'
 'Firestorm' 'Flower / Flourish' 'Fry' 'Ghitu Fire' 'Glint Hawk'
 'Goblin Raider' 'Grapeshot' 'Infest' 'Inkmoth Nexus' 'Masticore' 'Opt'
 'Pillage' 'Probe' 'Purge' 'Rebound' 'Recall' 'Scald' 'Shatter' 'Sleep'
 'Smelt' 'Smother' 'Spite / Malice' 'Splinter' 'Submerge' 'Tangle'
 'Thwart' 'Tinker' 'Tithe' 'Treachery' 'Tundra' 'Turn / Burn' 'Undermine'
 'Unknown Card' 'Upheaval' 'Vesuva' 'Void' 'Wasteland' 'Wonder']



in database, but not in unique
['Absorb Vis' 'Akki Underminer' "Aladdin's Lamp" 'Aven Mindcensor'
 'Azamuki, Treachery Incarnate' 'Belfry Spirit' 'Bone Splinters'
 'Cataclysmic Gearhulk' 'Crosis, the Purger' "Dispeller's Capsule"
 'Eliminate the Competition' 'El